<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/SalesMail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 4.6MB/s 


In [3]:
import MeCab

tagger = MeCab.Tagger("-Owakati")
sample = "今日は天気が良いですね"

print(tagger.parse(sample))

今日 は 天気 が 良い です ね 

